In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 球種選擇

In [17]:
hittype = 0 # 高遠球:0, 挑球:1

if hittype == 0:
  timeLenth = 60
  hittype_name = "high"

elif hittype == 1:
  timeLenth = 50
  hittype_name = "short"

else:
  print("error hittype")


# cut function

In [18]:
import os
import numpy as np
import csv

def slope(input_data):
    slope_col = np.zeros(len(input_data),dtype=float)
    for i in range(len(input_data) - 1):
        for j in range(1,4):
            temp = input_data[i+1][j] - input_data[i][j]
            slope_col[i] += temp**2
        slope_col[i] = slope_col[i]**0.5
    return slope_col

def ma(slope_col, n):
    slope_ma = np.zeros(len(slope_col),dtype=float)
    for i in range(n,len(slope_col)-n):
        for j in range(-n,n+1):
            slope_ma[i] += slope_col[i+j]
        slope_ma[i] /= float(2*n+1)
    return slope_ma

def data_cut(input_data, save_data, hit_type):
    slope_col = slope(input_data)
    slope_avg = np.average(slope_col)
    # n = 4
    if hit_type == "high":
        n = 6
    elif hit_type == "short":
        n = 5
    slope_ma = ma(slope_col, n)

    for i in range(60,len(slope_col)-50):

        if ( slope_ma[i] > slope_avg ) and ( slope_col[i]==max(slope_col[i-50:i+50]) ) : # 找到可能峰值
            start = 0  # 向前&向後找起點
            end = 0
            while i+start > (50+n) :
                start -= 1
                if slope_ma[i+start] <= slope_avg:
                    break
            while i+end < (len(slope_col)-50-n) :
                end += 1
                if slope_ma[i+end] <= slope_avg:
                    break

            if  hit_type == "high" and end-start > 40 and slope_ma[i] > 15: # 長遠球
                save_data.append(input_data[i-45:i+15, [1,2,3,5,6,7]])

            elif hit_type == "short" and end-start > 20 and slope_ma[i] > 15: # 挑球
                save_data.append(input_data[i-35:i+15, [1,2,3,5,6,7]])

# Removes null bytes from the input file and returns a sanitized version of the file.
def sanitize_file(input_file_path):
    sanitized_content = ""
    with open(input_file_path, 'r', encoding='utf-8', errors='replace') as f:
        content = f.read()
        sanitized_content = content.replace('\x00', '')

    return sanitized_content

In [19]:
# 指定資料集所在的路徑
data_path =  "/content/drive/MyDrive/資科羽球專題/data(.txt)"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt') and hittype_name in file_name and len(file_name)>=30]

labels = np.empty((0, 5), dtype=int)
data = np.empty((0, timeLenth, 6), dtype=float)

for t, input_filename in enumerate(txt_files):

    print(input_filename)

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitized_content = sanitize_file(input_file_path)

    lines = sanitized_content.split('\n')

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)

    save_data = []

    data_cut(input_data, save_data, hittype_name)

    save_data = np.array(save_data).astype(float)

    data = np.concatenate((data, save_data), axis=0)

    label_values = np.array([int(ch) for ch in input_filename[6:11]])  # 轉換為整數陣列

    label_values = label_values[np.newaxis, :]  # 增加一個維度以使其成為二維陣列

    for i in range(len(save_data)):
        labels = np.vstack((labels, label_values))

print(data.shape)
print(labels.shape)

h01_n_23342_110358030_high3.txt
h02_n_11122_101108044_high.txt
h03_y_55434_109703024_high.txt
h05_y_11111_110703043_high.txt
h06_y_33222_110302059_high.txt
h07_y_23221_109703056_high.txt
h08_y_22111_109703034_high.txt
h10_y_24244_109703003_high.txt
h11_n_22122_109540020_high.txt
h12_y_45543_109507014_high.txt
h13_y_11111_109305061_high.txt
h14_y_22121_109206049_high.txt
h15_y_33321_109101023_high.txt
h16_y_22111_109703018_high.txt
h09_y_23122_109703019_high.txt
(406, 60, 6)
(406, 5)


In [20]:
# 指定資料集所在的路徑
data_path =  "/content/drive/MyDrive/資科羽球專題/data(.txt)/1026"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt') and hittype_name in file_name and len(file_name)>=30]

for t, input_filename in enumerate(txt_files):

    print(input_filename)

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitized_content = sanitize_file(input_file_path)

    lines = sanitized_content.split('\n')

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)

    save_data = []

    data_cut(input_data, save_data, hittype_name)

    save_data = np.array(save_data).astype(float)

    data = np.concatenate((data, save_data), axis=0)

    label_values = np.array([int(ch) for ch in input_filename[6:11]])  # 轉換為整數陣列

    label_values = label_values[np.newaxis, :]  # 增加一個維度以使其成為二維陣列

    for i in range(len(save_data)):
        labels = np.vstack((labels, label_values))

# 指定資料集所在的路徑
data_path =  "/content/drive/MyDrive/資科羽球專題/data(.txt)/1031"

# 取得該路徑下所有的檔案名稱
all_files = os.listdir(data_path)

# 過濾出所有的 .txt 檔案
txt_files = [file_name for file_name in all_files if file_name.endswith('.txt') and hittype_name in file_name and len(file_name)>=30]

for t, input_filename in enumerate(txt_files):

    print(input_filename)

    input_file_path = os.path.join(data_path, input_filename)

    # Sanitize the file by removing null bytes
    sanitized_content = sanitize_file(input_file_path)

    lines = sanitized_content.split('\n')

    # Starting from the last line, move upwards until a complete line (with 7 commas) is found
    while lines and lines[-1].count(",") != 7:
        lines = lines[:-1]

    # Load the (potentially modified) data into a numpy array
    input_data = np.loadtxt(lines, delimiter=",", dtype=float)

    save_data = []

    data_cut(input_data, save_data, hittype_name)

    save_data = np.array(save_data).astype(float)

    data = np.concatenate((data, save_data), axis=0)

    label_values = np.array([int(ch) for ch in input_filename[6:11]])  # 轉換為整數陣列

    label_values = label_values[np.newaxis, :]  # 增加一個維度以使其成為二維陣列

    for i in range(len(save_data)):
        labels = np.vstack((labels, label_values))

print(data.shape)
print(labels.shape)

h08_y_23233_102611108_high.txt
h07_y_12112_102611107_high.txt
h09_y_34432_102611109_high.txt
h10_y_12111_102611110_high.txt
h11_y_12122_102611111_high.txt
h12_y_22111_102611112_high.txt
h13_y_23121_102611113_high.txt
h01_y_55555_102611101_high.txt
h02_y_45554_102611102_high.txt
h03_y_55555_102611103_high.txt
h04_y_55555_102611104_high.txt
h05_y_55555_102611105_high.txt
h06_y_55444_102611106_high.txt
h11_y_55555_103100011_high.txt
h09_y_55555_103100009_high.txt
h08_y_55554_103100008_high.txt
h07_y_55555_103100007_high.txt
h06_y_55545_103100006_high.txt
h05_y_55555_103100005_high.txt
h10_y_55555_103100010_high.txt
h01_y_55555_103100001_high.txt
h02_y_44444_103100002_high.txt
h03_y_22233_103100003_high.txt
h04_y_55454_103100004_high.txt
(848, 60, 6)
(848, 5)


In [21]:
from sklearn.model_selection import train_test_split
#定義訓練集和測試集的比例，例如訓練集佔 80%，測試集佔 20%
train_ratio = 0.8
test_ratio = 0.2
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=test_ratio, random_state=24)

print(x_train.shape)
print(x_test.shape)

(678, 60, 6)
(170, 60, 6)


In [22]:
x_train = x_train.reshape(x_train.shape)
x_test = x_test.reshape(x_test.shape)

In [23]:
class MyDataset:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import SimpleRNN, Dense, Input, Dropout

# 自定義輸出函數
def custom_output(x):
    return x

# 自定義損失函數
def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return loss

# 自定義準確性函數
def custom_accuracy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.abs(y_true - y_pred) < 0.5, tf.float32))

class CustomRNNModel(Model):
    def __init__(self, units, output_dim, num_layers):
        super(CustomRNNModel, self).__init__()
        self.rnn_layers = [SimpleRNN(units, return_sequences=(i < num_layers - 1)) for i in range(num_layers)]
        self.dense = Dense(output_dim)

    def call(self, inputs):
        x = inputs
        for rnn_layer in self.rnn_layers:
            x = rnn_layer(x)
        output = self.dense(x)
        output = custom_output(output)
        return output

# 使用MyDataset類
train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

# 使用tf.data來創建數據集對象
train_tfdata = tf.data.Dataset.from_tensor_slices((train_dataset.data, train_dataset.labels))
test_tfdata = tf.data.Dataset.from_tensor_slices((test_dataset.data, test_dataset.labels))

# 實例化自定義模型
model = CustomRNNModel(units=64, output_dim=5, num_layers=3)

# 超參數
learning_rate = 0.01
batch_size = 20
numEpoch = 20

# 編譯模型時指定超參數
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=custom_loss,
              metrics=[custom_accuracy])

# 訓練模型時指定超參數
model.fit(x_train, y_train, epochs=numEpoch, batch_size=batch_size)


# 評估模型
loss, accuracy = model.evaluate(test_tfdata.batch(8))
print(f'Loss: {loss}, Custom Accuracy: {accuracy}')

Epoch 1/20
34/34 [==============================] - 5s 36ms/step - loss: 4.7390 - custom_accuracy: 0.2000
Epoch 2/20
34/34 [==============================] - 1s 35ms/step - loss: 2.2817 - custom_accuracy: 0.2498
Epoch 3/20
34/34 [==============================] - 1s 36ms/step - loss: 2.2639 - custom_accuracy: 0.2506
Epoch 4/20
34/34 [==============================] - 1s 36ms/step - loss: 3.1087 - custom_accuracy: 0.2352
Epoch 5/20
34/34 [==============================] - 1s 36ms/step - loss: 1.7134 - custom_accuracy: 0.2927
Epoch 6/20
34/34 [==============================] - 1s 38ms/step - loss: 1.5573 - custom_accuracy: 0.3102
Epoch 7/20
34/34 [==============================] - 1s 35ms/step - loss: 1.4862 - custom_accuracy: 0.3058
Epoch 8/20
34/34 [==============================] - 2s 57ms/step - loss: 1.3979 - custom_accuracy: 0.3418
Epoch 9/20
34/34 [==============================] - 2s 68ms/step - loss: 1.4817 - custom_accuracy: 0.3637
Epoch 10/20
34/34 [===========================